# Quickstart

This notebook serves as a brief overview for getting started with CIMS. For this notebook to run properly, ensure:
* CIMS is downloaded and installed on your local machine according to the [installation instructions](../../docs/Installation.md)
* The `CIMS_env` conda environment has been activated according the [conda environment instructions](../../docs/WorkingWithCondaEnvironment.md)


## Import CIMS  & other packages

In [ ]:
import CIMS
import pprint as pp

## Model Validator

Now that we have loaded the `CIMS` module we can start to use it. First, we will validate our model description using the `ModelValidator` class.

In [ ]:
model_description_file = '../model_descriptions/CIMS_model_description_ALL_value.xlsb'
model_validator = CIMS.ModelValidator(model_description_file, 
                                        sheet_map={'model': 'Lists', 
                                                   'default_tech': 'Technology_Node templates'}, 
                                        node_col='Node')

model_validator.validate(verbose=True, raise_warnings=False)

For more information on using the `ModelValidator` class, checkout the [model validation tutorial](model_validation_instructions.ipynb). 

## Build the Model
There are two steps to building the model. First, we need to create a `ModelReader` object. This object reads in the model description Excel file and does some processessing to prepare the description to become a model. Then, this `ModelReader` object becomes the input to the all important `Model` object.

In [ ]:
# Model Reader
model_reader = CIMS.ModelReader(infile=model_description_file,
                                  sheet_map={'model': 'Lists',
                                             'default_tech': 'Technology_Node templates'},
                                  node_col='Node')

In [ ]:
# Model
model = CIMS.Model(model_reader)

## Run the CIMS Model

In [ ]:
model.run(max_iterations=5, show_warnings=False)

## Examine the Results

#### How has a fuel's price changed over the years?
This cell will print out the LCC for each year for the specified fuel. The `fuel_of_interest` variable should be the path to the fuel node of interest. 

In [ ]:
fuel_of_interest = 'CIMS.Canada.Alberta.Electricity' # Change this line to see how other fuels change

print("----- {} -----".format(fuel_of_interest))
for year in model.years:
    lcc = model.get_param('Life Cycle Cost', fuel_of_interest, year)
    print('{}: {}'.format(year, round(lcc, 2)))


#### How has marketshare changed between years?
This cell will print out the market shares for each technology for each within the specified node. The `node_of_interest` variable should be the path to the node of interest. 

In [ ]:
node_of_interest = 'CIMS.Canada.Alberta.Residential.Buildings.Floorspace.Lighting' # Change this line to see other nodes

# Printing the Table Header
headers = ('Technology', 'New Market Share', 'Total Market Share')
dash = '-' * 70
cols = '{:<20}{:>25}{:>25}'
print("{}".format(node_of_interest))
print(dash)
print(cols.format(*headers))             
print(dash)

# Printing the Table Content
total_stocks = {}
for year in model.years:
    print(year)
    for tech in model.graph.nodes[node_of_interest][year]['technologies']:
        n_ms = model.get_param('new_market_share', node_of_interest, year, tech)
        t_ms = model.get_param('total_market_share', node_of_interest, year, tech)
        print(cols.format(tech, f"{round(n_ms*100)}%", f"{round(t_ms*100)}%"))             
    print(dash)



#### How to find a Node's Path
This cell retrieves all of the nodes within the graph whose name contains the specified `node_name` . A Node's name is the last element in path, where elements are seperated by `.`. 

In [ ]:
node_name = 'Furnace'

for n in model.graph.nodes:
    n_name = n.split('.')[-1]
    if node_name in n_name:
        print(n)
